In [ ]:
# Ensure we use OTEL tracing.

import os

os.environ["TRULENS_OTEL_TRACING"] = "1"

In [ ]:
# Set up python resolution paths.

from pathlib import Path
import sys

# Add base dir to path to be able to access test folder.
base_dir = Path().cwd().parent.parent.resolve()
if str(base_dir) not in sys.path:
    print(f"Adding {base_dir} to sys.path")
    sys.path.append(str(base_dir))

In [ ]:
# Set up logging.

import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
handler.addFilter(logging.Filter("trulens"))
handler.setFormatter(formatter)
root.addHandler(handler)

In [ ]:
# Create snowpark session.
import os

from snowflake.snowpark import Session
from trulens.connectors.snowflake import SnowflakeConnector

snowflake_connection_parameters = {
    "account": os.environ["SNOWFLAKE_ACCOUNT"],
    "user": os.environ["SNOWFLAKE_USER"],
    "password": os.environ["SNOWFLAKE_USER_PASSWORD"],
    "database": os.environ["SNOWFLAKE_DATABASE"],
    "schema": os.environ["SNOWFLAKE_SCHEMA"],
    "role": os.environ["SNOWFLAKE_ROLE"],
    "warehouse": os.environ["SNOWFLAKE_WAREHOUSE"],
}


sf_connector = SnowflakeConnector(**snowflake_connection_parameters)

In [ ]:
snowpark_session = Session.builder.configs(
    snowflake_connection_parameters
).create()

In [ ]:
# Create TruSession.

from trulens.core.session import TruSession

tru_session = TruSession(connector=sf_connector)

In [ ]:
# Define app.

from random import random

from trulens.core.otel.instrument import instrument
from trulens.otel.semconv.trace import SpanAttributes


class TestApp:
    # TODO Not technically the right way to pass ground truth output, but using it as a workaround for this bug bash.
    @instrument(
        attributes=lambda ret, exception, *args, **kwargs: {
            SpanAttributes.RECORD_ROOT.GROUND_TRUTH_OUTPUT: args[2],
        },
    )
    def query(self, query: str, ground_truth_output: str) -> str:
        retrieved_contexts = self.get_contexts(query)
        return self.generation(query, retrieved_contexts)

    @instrument(
        span_type=SpanAttributes.SpanType.RETRIEVAL,
        attributes=lambda ret, exception, *args, **kwargs: {
            SpanAttributes.RETRIEVAL.QUERY_TEXT: args[1],
            SpanAttributes.RETRIEVAL.NUM_CONTEXTS: len(ret),
            SpanAttributes.RETRIEVAL.RETRIEVED_CONTEXTS: ret,
        },
    )
    def get_contexts(self, query: str) -> list[str]:
        return [
            "Seattle is a city in Washington",
            "Yakima is a town in Washington State",
        ]

    @instrument(
        span_type=SpanAttributes.SpanType.GENERATION,
    )
    def generation(self, query: str, contexts: list[str]) -> str:
        return "Yes" if random() < 0.5 else "No"

In [ ]:
# Create TruLens instrumented app from custom app.

from datetime import datetime

from trulens.apps.app import TruApp

APP_NAME = f"{os.getlogin()} pupr e2e {datetime.now().strftime('%Y%m%d%H%M%S')}".upper()
APP_VERSION = "V2"

test_app = TestApp()
tru_app = TruApp(
    test_app,
    app_name=APP_NAME,
    app_version=APP_VERSION,
    connector=sf_connector,
    main_method=test_app.query,
)

In [ ]:
print(tru_app.snowflake_object_type)  # EXTERNAL AGENT
print(tru_app.snowflake_object_name)  # APP_NAME
print(tru_app.snowflake_object_version)  # APP_VERSION

version_df = tru_app.snowflake_app_dao.list_agent_versions(APP_NAME)
print(version_df)

## Add runs to agent

In [ ]:
from trulens.core.run import Run
from trulens.core.run import RunConfig

run_name = "test_run_0225_bugbash_new".upper()

run_config = RunConfig(
    run_name=run_name,
    description="desc",
    dataset_name="My test dataframe name",
    source_type="DATAFRAME",
    label="label",
    dataset_spec={
        "input": "custom_input",
        "input_2": "ground_truth_output",
    },
)  # type: ignore

run: Run = tru_app.add_run(run_config=run_config)

In [ ]:
run = tru_app.get_run(run_name=run_name)

In [ ]:
run.describe()

In [ ]:
tru_app.list_runs()

### Start the Run (a pandas DataFrame or rows in user's table.) to invoke user's app directly and start ingestion

#### here we will be using a user provided test dataframe

In [ ]:
import csv
import random as rand

import pandas as pd

test_data_entries = []
with open("./test_data/ms_marco_hard_neg_balanced.csv", "r") as f:
    reader = csv.reader(f)
    next(reader, None)  # skip the headers
    for row in reader:
        test_data_entries.append({
            "custom_input": row[0],
            "ground_truth_output": row[1],
        })


rand.seed(42)
user_input_data_df = pd.DataFrame(rand.sample(test_data_entries, 100))

In [ ]:
run.start(input_df=user_input_data_df)

In [ ]:
run.describe().run_metadata

In [ ]:
run.get_status()

In [ ]:
run.compute_metrics([
    "coherence",
    "answer_relevance",
    "context_relevance",
    "groundedness",
])

In [ ]:
# run.cancel()  #TODO not implemented yet